### Co poprawic
    1. Czy mamy hot one encoded ?
    2. Rozmiar obrazkow jakie powinny byc?
        - 224x224x3
        - Ale get_batches z vgg juz zmienia wielkosc obrazko
    3. w katalogu dev mamy mniejsza probke zobazcmy jak zadziala
    

### Obliczenie sredniej


In [1]:
path_mean = "data/painter-by-numbers/train/"

In [2]:
import glob
import numpy as np
from scipy import misc

In [4]:
filelist = glob.glob(path_mean+"**/*.jpg")
mean_elements_counts = len(filelist)
mean_elements_counts

64633

In [5]:
all_imgs = np.array([misc.imread(fname) for fname in filelist])

In [6]:
means = [np.mean(img, axis=(0, 1)) for img in all_imgs]

In [7]:
means_sum = np.sum(means,axis=0)

In [8]:
means_final = means_sum/mean_elements_counts

In [9]:
means_final

array([ 132.56958886,  118.85338536,  103.05348692])

### Przygotowanie danych

In [16]:
path_train = "data/painter-by-numbers/train/"
path_valid = "data/painter-by-numbers/valid/"

In [17]:
path_info = "data/painter-by-numbers/train_info.csv"

In [18]:
import os, json
from glob import glob
import numpy as np

In [19]:
g = glob(path_train+"*.jpg")

In [20]:
len(g)

0

In [6]:
np.random.shuffle(g)

In [7]:
valid_files=g[:14800]

In [9]:
valid_files[:3]

['data/painter-by-numbers/train/51258.jpg',
 'data/painter-by-numbers/train/48123.jpg',
 'data/painter-by-numbers/train/78678.jpg']

In [10]:
pwd

u'/home/ubuntu/courses/deeplearning1/nbs'

In [11]:
if not os.path.exists(path_valid):
    os.mkdir(path_valid)
for f in valid_files:
    nf = f.replace("/train","/valid")
    os.rename(f,nf)
    

In [13]:
import csv
def move_files_to_categories(base_path):
    with open(path_info, 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        next(spamreader)
        for row in spamreader:
            artist = row[1]
            fileName = row[0]
            if not os.path.exists(base_path+artist):
                os.mkdir(base_path+artist)
            if os.path.exists(base_path+fileName):
                os.rename(base_path+fileName,base_path+artist+"/"+fileName)

In [14]:
move_files_to_categories(path_train)
move_files_to_categories(path_valid)

In [26]:
sample_path_train = "data/painter-by-numbers/sample/train/"
sample_path_valid = "data/painter-by-numbers/sample/valid/"
if not os.path.exists("data/painter-by-numbers/sample/"):
    os.mkdir("data/painter-by-numbers/sample/")
if not os.path.exists(sample_path_train):
    os.mkdir(sample_path_train)
if not os.path.exists(sample_path_valid):
    os.mkdir(sample_path_valid)

### Szkolenie

In [1]:
%matplotlib inline

In [2]:
#path="data/painter-by-numbers/sample/"
path="data/painter-by-numbers/dev/"
#path="data/painter-by-numbers/"

In [3]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

In [4]:
import utils; reload(utils)
from utils import plots


Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [5]:
# batch_size=64
batch_size=64

In [6]:
import vgg16_Painters;reload(vgg16_Painters)
from vgg16_Painters import Vgg16

In [7]:
vgg=Vgg16()

In [8]:
train_batch = vgg.get_batches(path+"train",batch_size=batch_size)
valid_batch = vgg.get_batches(path+"valid",batch_size=batch_size)

Found 172 images belonging to 6 classes.
Found 38 images belonging to 6 classes.


In [9]:
#keras zwraca kategoria jako klasy a my chcemy one hot encoded
train_classes = train_batch.classes
valid_classes = valid_batch.classes


In [10]:
train_labels = utils.onehot(train_classes)
valid_labels = utils.onehot(valid_classes)


In [11]:
len(vgg.model.layers)

38

In [12]:
vgg.finetune(train_batch,2)

ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false
ustawiam trainable na false


In [13]:
vgg.model.optimizer.lr = 0.00001

In [17]:
#Notice we are passing in the validation dataset to the fit() method
#For each epoch we test our model against the validation set
no_of_epochs=5
results_path=path+"results/"
latest_weights_filename = "painters_weights_2epochs_30train.h5"
for epoch in range(no_of_epochs):
    print("Running epoch: %d" % epoch)
    vgg.fit(train_batch, valid_batch, nb_epoch=1)
    vgg.model.save_weights(results_path+latest_weights_filename)
print("Completed %s fit operations" % no_of_epochs)

Running epoch: 0
Epoch 1/1
172/172 [==============================] - 7s - loss: 0.5239 - acc: 0.8140 - val_loss: 1.0166 - val_acc: 0.7368
Running epoch: 1
Epoch 1/1
172/172 [==============================] - 7s - loss: 0.4177 - acc: 0.8547 - val_loss: 0.9884 - val_acc: 0.7368
Running epoch: 2
Epoch 1/1
172/172 [==============================] - 7s - loss: 0.3959 - acc: 0.8953 - val_loss: 0.9637 - val_acc: 0.7368
Running epoch: 3
Epoch 1/1
172/172 [==============================] - 7s - loss: 0.2826 - acc: 0.9302 - val_loss: 0.9512 - val_acc: 0.7368
Running epoch: 4
Epoch 1/1
172/172 [==============================] - 7s - loss: 0.3674 - acc: 0.8605 - val_loss: 0.9376 - val_acc: 0.7368
Completed 5 fit operations


In [52]:
vgg.model.save_weights('data/painter-by-numbers/moje_paintint_ft1_ver2.h5')

In [57]:
from IPython.display import FileLink
FileLink('data/painter-by-numbers/moje_paintint_ft1_ver1.h5')

/home/ubuntu/courses/deeplearning1/nbs/data/painter-by-numbers/moje_paintint_ft1_ver1.h5

In [20]:
vgg.fit(train_batch,valid_batch,nb_epoch=1)

Epoch 1/1
64633/64633 [==============================] - 1846s - loss: 7.7677 - acc: 0.2119 - val_loss: 7.4706 - val_acc: 0.2408


In [ ]:
vgg.model.save_weights('data/painter-by-numbers/moje_paintint_ft2_ver2.h5')

In [ ]:
vgg.fit(train_batch,valid_batch,nb_epoch=1)

Epoch 1/1
 1792/64633 [..............................] - ETA: 1483s - loss: 6.4442 - acc: 0.3013

In [ ]:
vgg.model.save_weights('data/painter-by-numbers/moje_paintint_ft3_ver2.h5')